In [ ]:
#!pip install folium

In [ ]:
#importing of packages
import pandas as pd
import folium
from folium.plugins import HeatMap
from random import randint, uniform

In [ ]:
import pandas as pd
import re

# 1. Clean Postal Codes
def clean_postal_codes(df, column_name):
    """Clean and validate postal codes."""
    def clean_postal(postal):
        if pd.isna(postal):
            return None
        # Remove leading/trailing spaces, convert to uppercase, and remove invalid chars
        postal = re.sub(r"[^A-Z0-9]", "", postal.upper().strip())
        # Add space in the correct position if format matches
        if re.match(r"^[A-Z]\d[A-Z]\d[A-Z]\d$", postal):
            return postal[:3] + " " + postal[3:]
        return None  # Invalid postal code

    df[column_name] = df[column_name].apply(clean_postal)
    return df

# 2. Map Postal Codes to Latitude and Longitude
def map_postal_codes(df, postal_code_col, postal_mapping_path):
    """Map postal codes to their latitude and longitude using a reference file."""
    # Load the postal code mapping file
    postal_mapping = pd.read_csv(postal_mapping_path, on_bad_lines='skip')

    
    # Merge with survey data
    df = df.merge(postal_mapping, how="left", left_on=postal_code_col, right_on="POSTAL_CODE")
    
    # Group by postal codes and coordinates, then count
    grouped = df.groupby([postal_code_col, "LATITUDE", "LONGITUDE"]).size().reset_index(name="count")
    return grouped

# Main Script
if __name__ == "__main__":
    # File paths
    fake_data_output = "fake_postal_codes.csv"
    postal_mapping_file = "CanadianPostalCodes202403.csv"  # Latitude/Longitude data
    cleaned_output_file = "cleaned_postal_data.csv"


    # Load Fake Data and Clean
    fake_data = pd.read_csv(fake_data_output)
    fake_data_cleaned = clean_postal_codes(fake_data, 'postal_code')

    # Map to Latitude/Longitude
    postal_summary = map_postal_codes(fake_data_cleaned, 'postal_code', postal_mapping_file)


In [ ]:
# Step 3: Create an interactive Folium map
map_center = [43.7315, -79.7604]  # Centered on Brampton, Ontario
brampton_map = folium.Map(location=map_center, zoom_start=11)

,postal_code,LATITUDE,LONGITUDE,count
0,L6P 2B9,43.771787,-79.707262,1
1,L6P 2G7,43.777513,-79.682147,1
2,L6P 2J6,43.768210,-79.714220,1
3,L6P 2M8,43.632735,-79.831816,1
4,L6P 2T6,43.800868,-79.704911,1
...,...,...,...,...
81,L7A 4C7,43.702276,-79.822901,1
82,L7A 4H7,43.702276,-79.822901,1
83,L7A 4M4,43.691293,-79.838062,1
84,L7A 4V9,43.713282,-79.841336,1


In [ ]:
# Step 4: Add Circle Markers to the map
for _, row in postal_summary.iterrows():
    folium.CircleMarker(
        location=[row['LATITUDE'], row['LONGITUDE']],
        radius=row['count'] * 2,  # Marker size based on count
        color='blue',
        fill=True,
        fill_opacity=0.5,
        popup=f"<b>Postal Code:</b> {row['postal_code']}<br><b>Count:</b> {row['count']}"
    ).add_to(brampton_map)


Interactive map saved as brampton_survey_map.html


In [ ]:
# Step 5: Save the map as an HTML file
map_filename = "brampton_survey_map.html"
brampton_map.save(map_filename)

In [ ]:

# Step 6: Create the interactive map
map_center = [43.7315, -79.7604]  # Centered on Brampton, Ontario
brampton_map = folium.Map(location=map_center, zoom_start=11)

In [ ]:
# Step 7: Add circle markers to the map
for _, row in aggregated_data.iterrows():
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=row['count'] * 2,  # Marker size based on count
        color='blue',
        fill=True,
        fill_opacity=0.5,
        popup=f"<b>Postal Code:</b> {row['postal_code']}<br><b>Count:</b> {row['count']}"
    ).add_to(brampton_map)

In [ ]:
# Save the interactive map
map_filename = "brampton_map.html"
brampton_map.save(map_filename)

print(f"Interactive map saved as {map_filename}")